In [41]:
from jupyter_plotly_dash import JupyterDash
import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
# import method from python module
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
#FIXME: remove hard-coding of username/password
username = "aacuser"
password = "aac"
shelter = AnimalShelter(username, password)

# must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Rescue Aniimals')

# Grazioso Salvare’s logo
image_filename = './images/companylogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # header
    html.Div(id='hidden-div', style={'display':'flex'}),
    html.Div(
        
        html.A(
            href='https://www.snhu.com',
            #href='https://github.com/katiestapleton/PyMongoClientServer',
            children=[
                html.Img(id='companyLogo',
                    alt="Grazioso Salvare’s logo",
                    style = {
                        'Height' : '25%',
                        'maxHeight' : '150px',
                        'width': 'auto',
                        'padding' : '10px',
                    },
                    src='data:image/png;base64,{}'.format(encoded_image.decode())
                )
            ]
        )  ,
        style={
            'border': '5px solid #BF0404',
            'backgroundColor':'#F2F2F2',
            'textAlign' : 'center'
        },
    ),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.H3(html.Center("Hitchhiker's Guide to the Galaxy: Towel Search and Rescue Animals")),
    html.Hr(),
    html.Hr(),
    # body
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    ),
    html.Hr(),
    # table to display animal data to user
    dt.DataTable(
        id='datatable-id',
        # upload data
        data=df.to_dict('records'),        
        
        # style cells
        style_cell={
            'whiteSpace': 'normal',
            'height': 'auto',
            'minWidth': '100px',
            'maxWidth': '200px',
            'padding': '5px 10px',
            'textAlign': 'left'
        },
        style_header={
            'textAlign': 'center',
            'backgroundColor':'#F2F2F2',
            'fontWeight': 'bold'          
        },
        
        # rows and columns
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        #column_selectable=False,
        #row_selectable=True,
        #row_deletable=True,
        
        # edit permission
        editable=False,
                
        # filter & sort
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
      
        # page setup
        page_action="native",
        page_current= 0,
        page_size= 25
        
    ),
    html.Br(),
    html.Hr(),
    # chart and geolocation map are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
    ]),
    html.Hr(),
    html.Br(),
    
    # footer  
    html.P("\"The answer to the ultimate question of life, the universe and everything is 42.\" -H2G2."),
    html.P("Developed by Galaxy Travel Guide Katie Stapleton")
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    print("placeholder")
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

# interactive geolocation map to display location of pets
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # HQ is Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ] 


#!!do not remove!!
app